## Create meeting minutes from an audio file

#### I downloaded some Denver City council meeting minutes from this dataset
#### meeting back
#### I've put them in my Google Drive. The goal of this product is to use the Audio to generate meeting minutes, including actions.

In [1]:
%pip install requests torch bitsandbytes transformers sentencepiece accelerate openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports
import os
import requests
import torch
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from huggingface_hub import login
from openai import OpenAI
from IPython.display import display, Markdown, update_display

ImportError: dlopen(/opt/anaconda3/envs/llms/lib/python3.11/site-packages/torch/_C.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libgfortran.5.dylib
  Referenced from: <CAA510D4-816A-34E5-9021-485EF5D56159> /opt/anaconda3/envs/llms/lib/libopenblas.0.dylib
  Reason: tried: '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anac

In [3]:
# Constants
AUDIO_MODEL = "whisper-1"
LLAMA_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [4]:
# New Capability: connect this colab to your Google Drive
drive.mount('/content/drive')
audio_file_path = '/content/drive/MyDrive/audio.mp3'

NameError: name 'drive' is not defined

In [ ]:
# Sign in to Hugging Face

hf_token = userdata.get('hf_token')
if not hf_token:
    hf_token = input("Enter your Hugging Face token: ")
    userdata['hf_token'] = hf_token
login(token=hf_token, add_to_git_credential=True)
# New Capability: download and load the LLaMA model

In [ ]:
# Sign in to OpenAI using secrets in Colab

openai_api_key = userdata.get('OPEN_API_KEY')
openai = OpenAI(api_key=openai_api_key)
if not openai_api_key:
    openai_api_key = input("Enter your OpenAI API key: ")
    userdata['OPEN_API_KEY'] = openai_api_key

In [ ]:
# Use the Whisper model to transcribe the audio file to text

audio_file = open(audio_file_path, "rb")
transcription = openai.audio.transcriptions.create(
    file=audio_file,
    model=AUDIO_MODEL,
    response_format="text"
)
print(f"Transcription: {transcription}")

In [ ]:
system_message = "You are an assitant that produces minutes of meetings from the transcripts, with summary, key discussion points, takeaways, and action items for the owners."
user_prompt = f"Below is a transcription of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date: discussion points and action itms for the owners"
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL,
    quantization_config=quantization_config,
    device_map="auto",
    quantization_config=quantization_config,
)
outputs= model.generate(inputs, max_new_tokens = 2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))
# New Capability: save the minutes to a file

In [ ]:
# Gradio UI